In [11]:
### Import libraries ###

import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_predict, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, precision_score, accuracy_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

import matplotlib
import matplotlib.pyplot as plt

import pmdarima as pm
import statsmodels.api as api
from scipy.stats import norm

from utils import grab_data, import_wl_data, import_ph_data, import_ec_data, unit_scale, remove_outliers
from models.data_validation import ReportErrors

import warnings
warnings.filterwarnings('ignore')
plt.rc('font', size=12)

In [12]:
# read in metadata csv's #
depth120d = pd.read_csv("data/metadata - 120d.csv")
pH_meta = pd.read_csv("data/wq_metadata - pH.csv")
ec_meta = pd.read_csv("data/wq_metadata - EC.csv")

In [13]:
### Utility functions ###
def add_derv(df, tol = 0.00001):
    df = df.copy()
    df['smoothed'] = df['Value'].rolling(window=6*8, min_periods=1, center=True).mean()
    df['mean diff'] = df['smoothed'].diff(1).fillna(0) / df.index.to_series().diff(periods=1).dt.seconds #careful for non-USGS data
    df = df.drop(df[df['mean diff'] < -1e308]['mean diff'].index)
    df_down = df[df['mean diff'] < -1*tol]
    df_up = df[df['mean diff'] >= tol]
    
    return df, df_down, df_up

def cut_ends(df): # removes the 1st and 99th percentile value measurements to ensure no outliers
    try:
        q1, q99 = df['Value'].quantile([0.01, 0.99])
        a = df.copy()
        if len(df[df['Value']<q1]) < 0.02*len(df): 
            a = df[df['Value']>q1]
        if len(a[a['Value']>q99]) < 0.02*len(df):
            a = a[a['Value']<q99]
        return a
    except:
        return df

### Rule-based tests ###
def flag_depth_errors(df, offset):
    
    flag = False
    df, neg_flag, sat_flag = ReportErrors.detect_out_of_range(df, offset)
    df, und_flag = ReportErrors.detect_underliers(df)
    
    if neg_flag:
        flag = True
    if sat_flag:
        flag = True
    if und_flag:
        flag = True
    
    return df, flag

def flag_ph_errors(df, tol=25):
    
    flag = False
    initial_len = len(df)
    
    df = df.drop(df[df['Value']< 6.5].index)
    df = df.drop(df[df['Value'] > 11.0].index)
    
    if len(df) < initial_len - tol:
        flag = True
    
    return df, flag

def flag_ec_errors(df, tol=25):
    
    flag = False
    
    return df, flag

### Grid-search to optimize binary classification model ###
def svm_classification(data, y, scoring = 'accuracy'):
    params = [{'kernel':['linear', 'rbf', 'poly'], 'C':[0.1, 1, 10, 100]}]
    clf_pipe = SVC()
    gs_clf = GridSearchCV(clf_pipe, param_grid=params, scoring=scoring, cv=5)
    gs_clf.fit(data, y)
    return gs_clf.best_params_

In [14]:
def validate_depth(metadata, nbins=10, cv_k=5, dim=2, tol=1e-4, model='svm', scoring = 'accuracy'):
    '''
    Function accepts a metadata file and returns a binary classification prediction.
    Arguments
        - metadata: the metadata file (see data/ folder for examples)
        - nbins: an int or 2-value list defining the number of bins to use in the histogram
        - cv_k: the number of folds to use in the cross validation
        - dim: 1 (classify on value distributions) or 2 (classify on phase portraits)
        - tol: tolerance used in identifying which parts of the signal are in recession
        - model: the binary classification model used
        - scoring: the metric optimized on in the grid-search
    Returns: 
        - preditcted labels for every sample
        - the true value labels for every sample
    '''
    df = metadata.copy()
    df['flag'] = np.zeros(len(df)) 
    class_X = [] # list of X for classification
    class_i = [] # list of indicies to run classification on
    class_y = [] # list of true y for classification
        
    for row, col in df.iterrows():
        data = import_wl_data(node_id=col['Node ID'], site_id=col['Site ID']) # import timeseries
        data = data[col['start']:col['end']] # isolate the relevant subset
        label = col['Clean'] # get the true label value
        offset = col['Offset (mm)'] # get the offset (calibration parameter for water level sensor data)
        
        if len(data) == 0: # if theres no measurements, flag the timeseries
            df['flag'].iloc[row] = 1 
        else:
            data, flag = flag_depth_errors(data, offset) # this function needs to be specified to type of sensor
                
            if flag:
                df['flag'].iloc[row] = 1 # exclude any samples that were flagged by the rule-based tests
            else:
                depth, depth_down, depth_up = add_derv(data, tol=tol) # calculate the derivative
                if len(depth_down) == 0: # if no part of the timeseries is in recession, flag the sample
                    df['flag'].iloc[row] = 1
                else:
                    X = np.array(depth_down[['Value', 'mean diff']]) # calculate the phase portrait
                    if dim == 2:
                        h, x_e, y_e = np.histogram2d(X[:,0], X[:,1], nbins)
                    else:
                        h, x_e = np.histogram(X[:,0], nbins)
                    h = np.reshape(h, (-1,1))
                    mm = make_pipeline(StandardScaler(), MinMaxScaler()) # normalize the histogram
                    h = mm.fit_transform(h)
                    class_i.append(row) 
                    class_X.append(np.ndarray.flatten(h))
                    class_y.append(label)
    
    y = np.array(class_y) # prepare the classificiation dataset (all of the samples that were not flagged)
    X = np.array(class_X)

    # optimizes model parameters based on a gridsearch
    if model == 'mlp':
        best_params = mlp_classification(X, y, scoring=scoring)
        m = MLPClassifier(alpha=best_params['alpha'], 
                            random_state=best_params['random_state'],
                            hidden_layer_sizes = best_params['hidden_layer_sizes'],)
    elif model == 'svm':
        best_params = svm_classification(X, y, scoring=scoring) 
        m = SVC(kernel=best_params['kernel'], C=best_params['C'])
    elif model == 'nb':
        best_params = nb_classification(X, y, scoring=scoring)
        m = GaussianNB(var_smoothing=best_params['var_smoothing'])
    elif model == 'ada':
        best_params = ada_classification(X, y, scoring=scoring)
        m = AdaBoostClassifier(n_estimators=best_params['n_estimators'])
    elif model == 'gp':
        best_params = gp_classification(X, y, scoring=scoring)
        m = GaussianProcessClassifier(kernel=best_params['kernel'])
    elif model == 'knn':
        best_params = knn_classification(X, y, scoring=scoring)
        m = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
    else:
        return 0, -1
    
    y_pred = cross_val_predict(m, X, y, cv=cv_k) # use cross validation to get a prediction for every sample

    classified = pd.DataFrame({'y': y, 'prediction': y_pred})
    classified.index = class_i
    testdf = df[df['flag']==1]['Clean']
        
    flagged = pd.DataFrame({'y': testdf, 'prediction': np.zeros(len(testdf))})
    
    final_class = pd.concat([classified, flagged]) # re-combine the flagged and classified datasets
    
    return final_class['prediction'], final_class['y']

In [15]:
def validate_ph(metadata, nbins=10, cv_k = 5, dim=2, tol=1e-6, model='svm', scoring='accuracy'):
    '''
    Function accepts a metadata file and returns a binary classification prediction.
    Arguments:
        - metadata: the metadata file (see data/ folder for examples)
        - nbins: an int or 2-value list defining the number of bins to use in the histogram
        - cv_k: the number of folds to use in the cross validation
        - dim: 1 (classify on value distributions) or 2 (classify on phase portraits)
        - tol: tolerance used in identifying which parts of the signal are in recession
        - model: the binary classification model used
        - scoring: the metric optimized on in the grid-search
    Returns: 
        - preditcted labels for every sample
        - the true value labels for every sample
    '''
    y_predictions = np.zeros(len(metadata)) # define array of predictions
    classification_i = [] # list of indicies to run classification on
    classification_X = [] # list of X for classification
    classification_y = [] # list of true y for classification
    
    for row, col in metadata.iterrows():
        data = import_ph_data(col['node_id'], col['start'], col['end'])
        label = col['clean']

        data, flag = flag_ph_errors(data) # this function needs to be specified to type of sensor

        if len(data) != 0: 
            if not flag:
                if len(data) > 100: # for water quality data, trimming off the extreme value ends of the timeseries
                    data = cut_ends(data) # improves classification by ensuring all outliers are removed
                    
                ph, ph_down, ph_up = add_derv(data, tol=tol)

                if len(ph_down) != 0:
                    X = np.array(ph_down[['Value', 'mean diff']])
                    if dim ==2:
                        h, x_e, y_e = np.histogram2d(X[:,0], X[:,1], nbins)
                    else:
                        h, x_e = np.histogram(X[:,0], nbins)
                        
                    h = np.reshape(h, (-1,1))
                    mm = make_pipeline(StandardScaler(), MinMaxScaler())
                    h = mm.fit_transform(h)

                    classification_i.append(row)
                    classification_X.append(np.ndarray.flatten(h))
                    classification_y.append(label)
    
    X = np.array(classification_X)
    y = np.array(classification_y)
    
    if model == 'mlp':
        best_params = mlp_classification(X, y, scoring=scoring)
        m = MLPClassifier(alpha=best_params['alpha'], 
                            random_state=best_params['random_state'],
                            hidden_layer_sizes = best_params['hidden_layer_sizes'],)
    elif model == 'svm':
        best_params = svm_classification(X, y, scoring=scoring)
        m = SVC(kernel=best_params['kernel'], C=best_params['C'])
    elif model == 'nb':
        best_params = nb_classification(X, y, scoring=scoring)
        m = GaussianNB(var_smoothing=best_params['var_smoothing'])
    elif model == 'ada':
        best_params = ada_classification(X, y, scoring=scoring)
        m = AdaBoostClassifier(n_estimators=best_params['n_estimators'])
    elif model == 'gp':
        best_params = gp_classification(X, y, scoring=scoring)
        m = GaussianProcessClassifier(kernel=best_params['kernel'])
    elif model == 'knn':
        best_params = knn_classification(X, y, scoring=scoring)
        m = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
    else:
        return 0, -1
    
    y_pred = cross_val_predict(m, X, y, cv=cv_k)

    for i in range(len(classification_i)):
        y_predictions[classification_i[i]] = y_pred[i]
    print(y_predictions)
    return y_predictions, np.array(metadata['clean'])
                             

In [21]:
def validate_ec(metadata, nbins=10, cv_k=5, dim=2, tol=1e-6, model='svm', scoring='accuracy'):
    '''
    Function accepts a metadata file and returns a binary classification prediction.
    Arguments
        - metadata: the metadata file (see data/ folder for examples)
        - nbins: an int or 2-value list defining the number of bins to use in the histogram
        - cv_k: the number of folds to use in the cross validation
        - dim: 1 (classify on value distributions) or 2 (classify on phase portraits)
        - tol: tolerance used in identifying which parts of the signal are in recession
        - model: the binary classification model used
        - scoring: the metric optimized on in the grid-search
    Returns: 
        - preditcted labels for every sample
        - the true value labels for every sample
    '''
    y_predictions = np.zeros(len(metadata)) # define array of predictions
    classification_i = [] # list of indicies to run classification on
    classification_X = [] # list of X for classification
    classification_y = [] # list of true y for classification
    
    for row, col in metadata.iterrows():
        data = import_ec_data(col['node_id'], col['start'], col['end'])
        label = col['clean']

        data, flag = flag_ec_errors(data) # this function needs to be specified to type of sensor

        if len(data) != 0: 
            if not flag:
                if len(data) > 100:
                    data = cut_ends(data)
                    
                ec, ec_down, ec_up = add_derv(data, tol=tol)

                if len(ec_down) != 0:
                    X = np.array(ec_down[['Value', 'mean diff']])
                    if dim ==2:
                        h, x_e, y_e = np.histogram2d(X[:,0], X[:,1], nbins)
                    else:
                        h, x_e = np.histogram(X[:,0], nbins)
                    h = np.reshape(h, (-1,1))

                    mm = make_pipeline(StandardScaler(), MinMaxScaler())
                    h = mm.fit_transform(h)

                    classification_i.append(row)
                    classification_X.append(np.ndarray.flatten(h))
                    classification_y.append(label)
    
    
    X = np.array(classification_X)
    y = np.array(classification_y)
    if model == 'mlp':
        best_params = mlp_classification(X, y, scoring=scoring)
        m = MLPClassifier(alpha=best_params['alpha'], 
                            random_state=best_params['random_state'],
                            hidden_layer_sizes = best_params['hidden_layer_sizes'],)
    elif model == 'svm':
        best_params = svm_classification(X, y, scoring=scoring)
        m = SVC(kernel=best_params['kernel'], C=best_params['C'])
    elif model == 'nb':
        best_params = nb_classification(X, y, scoring=scoring)
        m = GaussianNB(var_smoothing=best_params['var_smoothing'])
    elif model == 'ada':
        best_params = ada_classification(X, y, scoring=scoring)
        m = AdaBoostClassifier(n_estimators=best_params['n_estimators'])
    elif model == 'gp':
        best_params = gp_classification(X, y, scoring=scoring)
        m = GaussianProcessClassifier(kernel=best_params['kernel'])
    elif model == 'knn':
        best_params = knn_classification(X, y, scoring=scoring)
        m = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
    else:
        return 0, -1
    
    y_pred = cross_val_predict(m, X, y, cv=cv_k)
    
    for i in range(len(classification_i)):
        y_predictions[classification_i[i]] = y_pred[i]
    
    return y_predictions, np.array(metadata['clean'])

In [86]:
def rolling_mean_check(metadata, dataset='wl', w='5D', nstd=2, pct_thrsh=0.05):
    '''
    Fits a moving average (MA) and moving stanrdard deviation to the time series to detect anomalies.
    Arguments
        - metadata: the metadata file (see data/ folder for examples)
        - dataset: 'wl', 'pH', or 'ec', used to specify which import and flagging functions should be used
        - w: window size, 'D' stands for days, used to specify the window size for the moving avg, std
        - nstd: the number of standard deviations from the mean a point has to be to be considered anomalous
        - pct_thrsh: the percentage of data that must be anomalous for a site to be classified as obstructed
    '''
    df = metadata.copy()
    df['flag'] = np.zeros(len(df))
    for row, col in df.iterrows():
        if dataset == 'wl':
            data = import_wl_data(node_id=col['Node ID'], site_id=col['Site ID'])
            data = data[col['start']:col['end']]
            offset = col['Offset (mm)']
        elif dataset == 'pH':
            data = import_ph_data(col['node_id'], col['start'], col['end'])
        elif dataset == 'ec':
            data = import_ec_data(col['node_id'], col['start'], col['end'])
        
        if len(data) == 0:
            df['flag'].iloc[row] = 1
        else:
            if dataset == 'wl':
                data, flag = flag_depth_errors(data, offset)
            elif dataset == 'pH':
                data, flag = flag_ph_errors(data)
            elif dataset == 'ec':
                data, flag = flag_ec_errors(data)
                
            if flag:
                df['flag'].iloc[row] = 1
            else:
                mean = data['Value'].rolling(window=w, min_periods=1, center=True).mean()
                std = data['Value'].rolling(window=w, min_periods=1, center=True).std()
                upper_thrsh = mean + nstd*std
                lower_thrsh = mean - nstd*std
                up_out = data[data['Value'] > upper_thrsh]
                low_out = data[data['Value'] < lower_thrsh]
                pct_out = len(up_out + low_out)/len(data)
                if pct_out > pct_thrsh:
                    df['flag'].iloc[row] = 1
    y_pred = np.ones(len(df)) - df['flag']
    
    if dataset == 'wl':
        return y_pred, df['Clean']
    else:
        return y_pred, df['clean']

In [82]:
def get_arima_fit(data):
    '''
    Automatically tunes and fits an ARIMA model to a timeseries, returning model predictons and residuals. 
    '''
    model = pm.auto_arima(np.array(data), seasonal=False, suppress_warnings=True, error_action="ignore")
    (p, d, q) = model.order
    model = api.tsa.SARIMAX(data, order=(p, d, q))
    model_fit = model.fit(disp=0, warn_convergence=False)
    residuals = pd.DataFrame(model_fit.resid)
    residuals[0][0] = 0
    predict = model_fit.get_prediction()
    predictions = pd.DataFrame(predict.predicted_mean)
    predictions['predicted_mean'][0] = data['Value'][0]
    return predictions, residuals

# its completely unfeasible to do a hyperparameter tuning for every instance
# at most lets try a few values for alpha, and maybe one or two values for window_sz
# this is going to take so long, yikes
def set_dynamic_threshold(residuals, window_sz=144, alpha=0.01, min_range=0.0):
    """
    set_dynamic_threshold determines a threshold for each point based on the local confidence interval
    considering the model residuals looking forward and backward a specified number of steps.
    Arguments:
        residuals: series like object or a data frame of model residuals.
        alpha: scalar between 0 and 1 representing the acceptable uncertainty.
        window_sz: integer representing how many data points to use in both directions.
            default = 144 for one day for 10-minute data.
    Returns:
        threshold: data frame of columns of low and high threshold values.
    """
    threshold = []  # initialize empty list to hold thresholds
    z = norm.ppf(1 - alpha / 2)

    # if the window size parameter is too big for this data set
    if (window_sz > len(residuals)):
        print("WARNING: in set_dynamic_threshold(), window_sz > len(data)! Reducing window_sz.")
        window_sz = len(residuals)  # reduce the window to the max allowable

    # loop through data and add each threshold pair
    for i in range(0, len(residuals)):
        if (window_sz > i):  # index is closer than window size to left edge of data
            lo = 0
        else:  # look back as far as the window size
            lo = i - window_sz
        if (i + window_sz > len(residuals)):  # index is close to right edge of data
            hi = len(residuals)
        else:  # look forward as far as the window size
            hi = i + window_sz

        # calculate the range of probable values using given alpha
        mean = np.mean(residuals[lo:(hi + 1)])[0]#.mean()
        sigma = np.std(residuals[lo:(hi + 1)])[0]#.std()
        th_range = z * sigma
        if (th_range < min_range):
            th_range = min_range
        # append pair of upper and lower thresholds
        threshold.append([mean - th_range, mean + th_range])

    threshold = pd.DataFrame(threshold, columns=['low', 'high'])

    return threshold

def outlier_count(data, window_sz=144):
    '''
    Given a timeseries, fits an ARIMA model, determines dynamic thresholds and then identifies outliers. 
    Returns a count of how many outliers there are. 
    '''
    predictions, residuals = get_arima_fit(data)
    threshold = set_dynamic_threshold(residuals, window_sz=window_sz)
    up_outliers = data[data['Value'] > predictions['predicted_mean'] + np.array(threshold['high'])]
    down_outliers = data[data['Value'] < predictions['predicted_mean'] + np.array(threshold['low'])]
    return len(up_outliers) + len(down_outliers)

def count_arima_outliers(metadata, dataset='wl', window_sz=144):
    '''
    Uses an ARIMA model to detect anomalies and determine if a site is obstructed or not. 
    Arguments:
        - metadata: the metadata file (see data/ folder for examples)
        - dataset: 'wl', 'pH', or 'ec', used to specify which import and flagging functions should be used
        - window_sz: integer representing how many data points to use in both directions.
    '''
    df = metadata.copy()
    df['outlier count'] = np.zeros(len(df))
    df['percent outlier'] = np.zeros(len(df))
    
    for row, col in df.iterrows():
        if dataset == 'wl':
            data = import_wl_data(node_id=col['Node ID'], site_id=col['Site ID'])
            data = data[col['start']:col['end']]
            offset = col['Offset (mm)']
        elif dataset =='pH':
            data = import_ph_data(col['node_id'], col['start'], col['end'])
        elif dataset == 'ec':
            data = import_ec_data(col['node_id'], col['start'], col['end'])
        
        if len(data) == 0:
            df['outlier count'].iloc[row] = len(data)
        else:
            if dataset == 'wl':
                data, flag = flag_depth_errors(data, offset)
            elif dataset == 'pH':
                data, flag = flag_ph_errors(data)
            elif dataset == 'ec':
                data, flag = flag_ec_errors(data)
                
            if flag:
                df['outlier count'].iloc[row] = len(data)
            else:
                df['outlier count'].iloc[row] = outlier_count(data[['Value']], window_sz=window_sz)
        df['percent outlier'].iloc[row] = df['outlier count'].iloc[row] / len(data)
        
        if row % 10 ==0:
            print(f"row {row} finished processing")
    return df

In [62]:
def runchecks_depth(metadata):
    '''
    Runs the rule-based tests for the water level dataset. 
    In this function, y_pred is 0 for all the flagged sites and 1 for all sites that were not flagged. 
    '''
    df = metadata.copy()
    df['flag'] = np.zeros(len(df))
    
    for row, col in df.iterrows():
        data = import_wl_data(node_id=col['Node ID'], site_id=col['Site ID'])
        data = data[col['start']:col['end']]
        offset = col['Offset (mm)']
        
        if len(data) == 0:
            df['flag'].iloc[row] = 1
        else:
            data, flag = flag_depth_errors(data, offset) # this function needs to be specified to type of sensor
            if flag:
                df['flag'].iloc[row] = 1
    y_pred = np.ones(len(df)) - df['flag']
    
    return y_pred, df['Clean']

def runchecks_pH(metadata):
    '''
    Runs the rule-based tests for the pH dataset. 
    In this function, y_pred is 0 for all the flagged sites and 1 for all sites that were not flagged. 
    '''
    df = metadata.copy()
    df['flag'] = np.zeros(len(df))
    
    for row, col in df.iterrows():
        data = import_ph_data(col['node_id'], col['start'], col['end'])
        
        if len(data) == 0:
            df['flag'].iloc[row] = 1
        else:
            data, flag = flag_ph_errors(data) # this function needs to be specified to type of sensor
            if flag:
                df['flag'].iloc[row] = 1
    y_pred = np.ones(len(df)) - df['flag']
    
    return y_pred, df['clean']

# Table 1

## Water level

In [16]:
# classification on phase portraits for water level data 
y_pred, y = validate_depth(depth120d, nbins=[10,4], cv_k=5, dim=2, model='svm', scoring = 'accuracy')
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.8387096774193549
precision: 0.8211382113821138
false positive rate: 0.11827956989247312


In [24]:
# classification on value distribution for water level data 
y_pred, y = validate_depth(depth120d, nbins=10, cv_k=5, dim=1, model='svm', scoring = 'accuracy')
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.8064516129032258
precision: 0.7829457364341085
false positive rate: 0.15053763440860216


In [48]:
# moving average for water level data 
windows = ['1D', '5D', '7D', '14D', '21D', '30D', '45D']
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
nstds = [2,3,4]
acc = []
for w in windows:
    for p in pct_thrshs:
        for n in nstds:
            y_pred, y = rolling_mean_check(depth120d, dataset='wl', w=w, pct_thrsh=p, nstd=n)
            C = confusion_matrix(y,y_pred)
            accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
            acc.append({'window': w, 'pct': p, 'nstd': n, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
acc_df = pd.DataFrame(acc)
acc_df = acc_df.sort_values(["accuracy", "precision"])

y_pred, y = rolling_mean_check(depth120d, dataset='wl', w=acc_df.iloc[-1]['window'], 
                                pct_thrsh=acc_df.iloc[-1]['pct'], nstd=acc_df.iloc[-1]['nstd'])
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.7956989247311828
precision: 0.7751937984496124
false positive rate: 0.15591397849462366


In [83]:
# ARIMA for water level data 
arima_120df = count_arima_outliers(depth120d, window_sz=288)
lengths = []
for row, col in depth120d.iterrows():
    data = import_wl_data(node_id=col['Node ID'], site_id=col['Site ID'])
    data = data[col['start']:col['end']]
    lengths.append(len(data))
arima_120df['percent outleirs'] = arima_120df['outlier count']/lengths
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
acc = []
for p in pct_thrshs:
    y_pred = [0 if i > p else 1 for i in arima_120df['percent outleirs']]
    y = arima_120df['Clean']
    C = confusion_matrix(y,y_pred)
    accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
    acc.append({'pct': p, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
acc_df = pd.DataFrame(acc)
acc_df = acc_df.sort_values(["accuracy", "precision"])

y_pred = [0 if i > acc_df.iloc[-1]['pct'] else 1 for i in arima_120df['percent outleirs']]
y = arima_120df['Clean']
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

row 0 finished processing
row 10 finished processing
row 20 finished processing
row 30 finished processing
row 40 finished processing
row 50 finished processing
row 60 finished processing
row 70 finished processing
row 80 finished processing
row 90 finished processing
row 100 finished processing
row 110 finished processing
row 120 finished processing
row 130 finished processing
row 140 finished processing
row 150 finished processing
row 160 finished processing
row 170 finished processing
row 180 finished processing
accuracy: 0.7634408602150538
precision: 0.7152317880794702
false positive rate: 0.23118279569892472


In [58]:
# Rules-based tests for water level data
y_pred, y = runchecks_depth(depth120d)
print(len(np.where(y_pred-y == 0)[0])/len(y))
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

0.7688172043010753
accuracy: 0.7688172043010753
precision: 0.72
false positive rate: 0.22580645161290322


## pH

In [18]:
# classification on phase portraits for pH data 
y_pred, y = validate_ph(pH_meta, nbins=[5, 4])
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
accuracy: 0.967741935483871
precision: 0.9672131147540983
false positive rate: 0.021505376344086023


In [67]:
# classification on value distribution for pH data 
y_pred, y = validate_ph(pH_meta, nbins=6, dim=1)
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
accuracy: 0.946236559139785
precision: 0.9365079365079365
false positive rate: 0.043010752688172046


In [64]:
# moving average for pH data 
windows = ['1D', '5D', '7D', '14D', '21D', '30D', '45D']
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
nstds = [2,3,4]
acc = []
for w in windows:
    for p in pct_thrshs:
        for n in nstds:
            y_pred, y = rolling_mean_check(pH_meta, dataset='pH', w=w, pct_thrsh=p, nstd=n)
            C = confusion_matrix(y,y_pred)
            accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
            acc.append({'window': w, 'pct': p, 'nstd': n, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
            acc_df = pd.DataFrame(acc)
            
acc_df = acc_df.sort_values(["accuracy", "precision"])
y_pred, y = rolling_mean_check(pH_meta, dataset='pH', w=acc_df.iloc[-1]['window'], 
                                pct_thrsh=acc_df.iloc[-1]['pct'], nstd=acc_df.iloc[-1]['nstd'])
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.8387096774193549
precision: 0.8082191780821918
false positive rate: 0.15053763440860216


In [63]:
y_pred, y = runchecks_pH(pH_meta)
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.8279569892473119
precision: 0.7894736842105263
false positive rate: 0.17204301075268819


In [84]:
arima_pH_df = count_arima_outliers(pH_meta, dataset='pH')
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
acc = []
for p in pct_thrshs:
    y_pred = [0 if i > p else 1 for i in arima_pH_df['percent outlier']]
    y = arima_pH_df['clean']
    C = confusion_matrix(y,y_pred)
    accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
    acc.append({'pct': p, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
acc_df = pd.DataFrame(acc)
acc_df = acc_df.sort_values('accuracy')

y_pred = [0 if i > acc_df.iloc[-1]['pct'] else 1 for i in arima_pH_df['percent outlier']]
y = arima_pH_df['clean']
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

row 0 finished processing
row 10 finished processing
row 20 finished processing
row 30 finished processing
row 40 finished processing
row 50 finished processing
row 60 finished processing
row 70 finished processing
row 80 finished processing
row 90 finished processing
accuracy: 0.8279569892473119
precision: 0.8055555555555556
false positive rate: 0.15053763440860216


## EC

In [22]:
# classification on phase portraits for ec data 
y_pred, y = validate_ec(ec_meta, nbins=[7,1])
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.9615384615384616
precision: 0.971830985915493
false positive rate: 0.02564102564102564


In [27]:
# classification on value distribution for ec data 
y_pred, y = validate_ec(ec_meta, nbins=7, dim=1)
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.9615384615384616
precision: 0.971830985915493
false positive rate: 0.02564102564102564


In [47]:
# moving average for ec data 
windows = ['1D', '5D', '7D', '14D', '21D', '30D', '45D']
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
nstds = [2,3,4]
acc = []
for w in windows:
    for p in pct_thrshs:
        for n in nstds:
            y_pred, y = rolling_mean_check(ec_meta, dataset='ec', w=w, pct_thrsh=p, nstd=n)
            C = confusion_matrix(y,y_pred)
            accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
            acc.append({'window': w, 'pct': p, 'nstd': n, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
acc_df = pd.DataFrame(acc)
acc_df = acc_df.sort_values(["accuracy", "precision"])
y_pred, y = rolling_mean_check(ec_meta, dataset='ec', w=acc_df.iloc[-1]['window'], 
                                pct_thrsh=acc_df.iloc[-1]['pct'], nstd=acc_df.iloc[-1]['nstd'])
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

accuracy: 0.9102564102564102
precision: 0.92
false positive rate: 0.07692307692307693


In [85]:
arima_ec_df = count_arima_outliers(ec_meta, dataset='ec')
pct_thrshs = [0.01, 0.05, 0.10, 0.15, 0.2, 0.25]
acc = []
for p in pct_thrshs:
    y_pred = [0 if i > p else 1 for i in arima_ec_df['percent outlier']]
    y = arima_ec_df['clean']
    C = confusion_matrix(y,y_pred)
    accuracy = len(np.where(y_pred-y == 0)[0])/len(y)
    acc.append({'pct': p, 'accuracy': accuracy, 'precision': C[1,1]/(C[1,1]+C[0,1])})
acc_df = pd.DataFrame(acc)
acc_df = acc_df.sort_values('accuracy')

y_pred = [0 if i > acc_df.iloc[-1]['pct'] else 1 for i in arima_ec_df['percent outlier']]
y = arima_ec_df['clean']
print('accuracy:', accuracy_score(y, y_pred))
print('precision:', precision_score(y, y_pred))
print('false positive rate:', confusion_matrix(y, y_pred)[0,1]/len(y))

row 0 finished processing
row 10 finished processing
row 20 finished processing
row 30 finished processing
row 40 finished processing
row 50 finished processing
row 60 finished processing
row 70 finished processing
accuracy: 0.8974358974358975
precision: 0.8974358974358975
false positive rate: 0.10256410256410256
